In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from itertools import combinations, groupby, chain
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
aisles = pd.read_csv(r'C:\Users\pramo\Documents\Pramod_Docs\Coursework\ADM\Final_Project_Proposal\data\aisles.csv')
departments = pd.read_csv(r'C:\Users\pramo\Documents\Pramod_Docs\Coursework\ADM\Final_Project_Proposal\data\departments.csv')
orders = pd.read_csv(r'C:\Users\pramo\Documents\Pramod_Docs\Coursework\ADM\Final_Project_Proposal\data\orders.csv')
order_products_prior = pd.read_csv(r'C:\Users\pramo\Documents\Pramod_Docs\Coursework\ADM\Final_Project_Proposal\data\order_products__prior.csv')
order_products_train = pd.read_csv(r'C:\Users\pramo\Documents\Pramod_Docs\Coursework\ADM\Final_Project_Proposal\data\order_products__train.csv')
products = pd.read_csv(r'C:\Users\pramo\Documents\Pramod_Docs\Coursework\ADM\Final_Project_Proposal\data\products.csv')

In [3]:
print(len(aisles))
print(len(departments))
print(len(orders))
print(len(order_products_prior))
print(len(order_products_train))
print(len(products))

134
21
3421083
32434489
1384617
49688


In [4]:
orders = orders.sample(n=2000000)
# order_products_prior = order_products_prior.sample(n=100000)
order_products_train = order_products_train.sample(n=1000000)
# products = products.sample(n=10000)

In [5]:
products_aisles = pd.merge(products, aisles, on = 'aisle_id', how = 'left')
products_desc = pd.merge(products_aisles, departments, on = 'department_id', how = 'left')
products_desc.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [6]:
products_desc[products_desc['aisle']=='missing']

,product_id,product_name,aisle_id,department_id,aisle,department
37,38,Ultra Antibacterial Dish Liquid,100,21,missing,missing
71,72,Organic Honeycrisp Apples,100,21,missing,missing
109,110,Uncured Turkey Bologna,100,21,missing,missing
296,297,"Write Bros Ball Point Pens, Cap-Pen, Medium (1...",100,21,missing,missing
416,417,Classics Baby Binks Easter Chocolate Bunny,100,21,missing,missing
...,...,...,...,...,...,...
49552,49553,Organic Plain Skyr,100,21,missing,missing
49574,49575,Chocolate Vanilla Malt Custard,100,21,missing,missing
49640,49641,"8\"" Pecan Pie",100,21,missing,missing
49663,49664,Lemon Cayenne Drinking Vinegar,100,21,missing,missing


In [7]:
pickle.dump(products_desc, open("products_desc.p", "wb"))

In [8]:
orders_test = orders[orders['eval_set']=='test']
orders_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3042807,1662878,183572,test,7,2,15,30.0
304038,1190768,18380,test,6,2,12,8.0
1159488,2511864,69672,test,100,4,16,2.0
111564,253606,6741,test,5,3,14,7.0
944705,3088878,56674,test,24,1,13,0.0
...,...,...,...,...,...,...,...
36969,339159,2288,test,6,0,11,6.0
2738409,2315803,164986,test,13,2,17,7.0
3277952,414440,197568,test,5,4,14,13.0
2863804,1084441,172777,test,18,3,15,19.0


In [9]:
# merged_order_products = pd.merge(order_products_train, order_products_prior, how = 'outer')
merged_order_products = order_products_train
merged_order_products.head()

,order_id,product_id,add_to_cart_order,reordered
401224,981402,41950,9,1
1361208,3363993,44232,1,0
541106,1331366,23543,28,0
1291314,3189879,5818,4,1
481197,1179872,11513,5,1


In [10]:
order_products_desc = pd.merge(order_products_train, products_desc, on = 'product_id')
order_products_desc.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,981402,41950,9,1,Organic Tomato Cluster,83,4,fresh vegetables,produce
1,3059999,41950,7,1,Organic Tomato Cluster,83,4,fresh vegetables,produce
2,2601611,41950,3,1,Organic Tomato Cluster,83,4,fresh vegetables,produce
3,1422535,41950,12,1,Organic Tomato Cluster,83,4,fresh vegetables,produce
4,1023679,41950,7,0,Organic Tomato Cluster,83,4,fresh vegetables,produce


In [11]:
merged_orders = pd.merge(orders, order_products_desc, on = 'order_id')
merged_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2360078,172226,train,24,1,7,30.0,45447,2,1,Gluten Free Covered Pretzels Yogurt,107,19,chips pretzels,snacks
1,2360078,172226,train,24,1,7,30.0,17721,6,1,Original Dairy Free Coconut Milk,91,16,soy lactosefree,dairy eggs
2,2360078,172226,train,24,1,7,30.0,26243,9,1,Graduates Banana Puffs,92,18,baby food formula,babies
3,2360078,172226,train,24,1,7,30.0,32950,8,1,"Breaded Chicken Breast Nuggets, Gluten Free",129,1,frozen appetizers sides,frozen
4,2360078,172226,train,24,1,7,30.0,24613,7,1,Sweet Potato Puffs Cereal Snack,92,18,baby food formula,babies


In [12]:
test = orders[orders['eval_set']=='test']
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3042807,1662878,183572,test,7,2,15,30.0
304038,1190768,18380,test,6,2,12,8.0
1159488,2511864,69672,test,100,4,16,2.0
111564,253606,6741,test,5,3,14,7.0
944705,3088878,56674,test,24,1,13,0.0


In [13]:
pickle.dump(merged_orders, open("merged_orders.p", "wb"))

In [14]:
user_data = merged_orders[['user_id', 'order_number', 'order_dow', 'order_hour_of_day','days_since_prior_order', 'aisle']]

In [15]:
user_data = pd.get_dummies(user_data, prefix=None, columns=['aisle'])

In [16]:
user_data.tail()

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
583153,158220,29,2,12,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
583154,146690,21,1,13,12.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
583155,100004,9,3,19,12.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
583156,100004,9,3,19,12.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
583157,100004,9,3,19,12.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
user_data.head()

,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
0,172226,24,1,7,30.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,172226,24,1,7,30.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,172226,24,1,7,30.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,172226,24,1,7,30.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,172226,24,1,7,30.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def reduce_memory(df):
    
    """
    This function reduce the dataframe memory usage by converting it's type for easier handling.
    
    Parameters: Dataframe
    Return: Dataframe
    """
    
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    
    for col in df.columns:
        if df[col].dtypes in ["int64", "int32", "int16"]:
            
            cmin = df[col].min()
            cmax = df[col].max()
            
            if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            
            elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            
            elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        
        if df[col].dtypes in ["float64", "float32"]:
            
            cmin = df[col].min()
            cmax = df[col].max()
            
            if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            
            elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
    
    print("")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    
    return df


Before memory reduce

In [19]:
mem_usg = user_data.memory_usage().sum() / 1024**2 
print("Memory usage is: ",mem_usg," MB")

Memory usage is:  101.21799087524414  MB


In [20]:
user_data = reduce_memory(user_data)
mem_usg = user_data.memory_usage().sum() / 1024**2 
print("Memory usage is: ",mem_usg," MB")

Memory usage of properties dataframe is : 101.21799087524414  MB

___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  83.97756385803223  MB
This is  82.96703296703296 % of the initial size
Memory usage is:  83.97756385803223  MB


In [21]:
# user_data = user_data.sample(n= 1000000)

In [22]:
# Splitting the data in thirds to use groupby then will rejoin them
user_data1 = user_data[user_data['user_id'] <= 65000]
user_data2 = user_data[(user_data['user_id'] <= 135000) & (user_data['user_id'] > 65000)]
user_data3 = user_data[user_data['user_id'] > 135000]

In [23]:
grouped_user1 = user_data1.groupby('user_id').sum()
grouped_user2 = user_data2.groupby('user_id').sum()
grouped_user3 = user_data3.groupby('user_id').sum()

In [24]:
user_data1_noaisles = user_data1.iloc[:,:5]
user_data2_noaisles = user_data2.iloc[:,:5]
user_data3_noaisles = user_data3.iloc[:,:5]

In [25]:
group1 = user_data1_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0],
                                                     'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group2 = user_data2_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0],
                                                     'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group3 = user_data3_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0],
                                                     'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})

In [26]:
grouped_user1['num_orders'] = group1.order_number
grouped_user2['num_orders'] = group2.order_number
grouped_user3['num_orders'] = group3.order_number

grouped_user1['mean_days_since'] = group1.days_since_prior_order
grouped_user2['mean_days_since'] = group2.days_since_prior_order
grouped_user3['mean_days_since'] = group3.days_since_prior_order

grouped_user1['mode_order_dow'] = group1.order_dow
grouped_user2['mode_order_dow'] = group2.order_dow
grouped_user3['mode_order_dow'] = group3.order_dow

grouped_user1['median_order_hour'] = group1.order_hour_of_day
grouped_user2['median_order_hour'] = group2.order_hour_of_day
grouped_user3['median_order_hour'] = group3.order_hour_of_day

grouped_user1.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user2.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user3.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)

grouped_users = pd.concat([grouped_user1, grouped_user2, grouped_user3], axis=0)
grouped_users.head()

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,...,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,num_orders,mean_days_since,mode_order_dow,median_order_hour
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,11,14.0,4,8.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,15,30.0,1,11.0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,6.0,0,11.0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,21,6.0,2,11.0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,6,30.0,0,19.0


In [27]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):

    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]

    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]

    return(df[seg1 + seg2 + seg3])

In [28]:
grouped_users = movecol(grouped_users,
                        cols_to_move=['num_orders', 'mode_order_dow', 'median_order_hour', 'mean_days_since'],
                        ref_col='aisle_fresh dips tapenades',
                        place='Before')
grouped_users.head()

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
pickle.dump(grouped_users, open("grouped_users.p", "wb"))

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_users = scaler.fit_transform(grouped_users)

In [31]:
from sklearn.cluster import KMeans
k_means_20 = KMeans(n_clusters=20, random_state=12, algorithm='full').fit(scaled_users)
pickle.dump(k_means_20, open("k_means_20.p", "wb"))

In [32]:
grouped_users['cluster'] = k_means_20.labels_

In [33]:
pickle.dump(grouped_users, open('clustered_users.p', 'wb'))

In [34]:
clustered_users = pickle.load(open("clustered_users.p", "rb"))

In [35]:
clustered_users.reset_index(inplace=True)

In [36]:
clustered_orders = merged_orders.merge(clustered_users[['user_id','cluster']], on = 'user_id')

In [37]:
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else:
        return pd.Series(Counter(iterable)).rename("freq")


# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

#Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]

        for item_pair in combinations(item_list, 2):
            yield item_pair


# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
            .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}),
                   left_on='item_A', right_index=True)
            .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}),
                   left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB',
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
             .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
             .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [38]:
def association_rules(order_item, min_support):

    #     print("Starting number of order_item pairs: {:22d}".format(len(order_item)))
    print("Starting number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                                  order_item.nunique(),len(order_item)))

    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item)


    # Filter from order_item items below min support
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    #     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaning number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                                  order_item.nunique(),len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    #     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaining number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                                   order_item.nunique(),len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item)


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)

    item_pairs['confAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])


    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [39]:
cluster_orders = {}
for x in range(0,20):
    i = clustered_orders[clustered_orders['cluster'] == x]
    i = i[['order_id', 'product_id']]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [40]:
group_association_rules_dic = {}
for x in range(0,20):
    print('Cluster: ', x)
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Cluster:  0
Starting number of orders, items, order_item pairs: 1690, 5491, 19711
Items with support >= 0.0001:            5491
Remaning number of orders, items, order_item pairs: 1690, 5491, 19711
Remaining orders with 2+ items:        1677
Remaining number of orders, items, order_item pairs: 1677, 5491, 19698
Item pairs:                          121473
Item pairs with support >= 0.0001:     121473

Cluster:  1
Starting number of orders, items, order_item pairs: 23509, 16853, 106358
Items with support >= 0.0001:            6665
Remaning number of orders, items, order_item pairs: 23057, 6665, 93303
Remaining orders with 2+ items:       19234
Remaining number of orders, items, order_item pairs: 19234, 6665, 89480
Item pairs:                          172247
Item pairs with support >= 0.0001:      17952

Cluster:  2
Starting number of orders, items, order_item pairs: 450, 2617, 3863
Items with support >= 0.0001:            2617
Remaning number of orders, items, order_item pairs: 450, 2617

In [41]:
# format item association rule dataframes and merge product names

for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
                                                                          .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
                                                                          .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                     'product_name_B','freqAB','supportAB','freqA',
                                                                     'supportA','freqB','supportB','confAtoB',
                                                                     'confBtoA','lift']]

In [42]:
pickle.dump(group_association_rules_dic, open("grouped_association_rules_dic.p", "wb"))

In [43]:
def product_to_product(cluster, product_id, item_lift, product_name, num_products):
    df = group_association_rules_dic[cluster]
    df = df[(df['item_A'] == product_id) | (df['item_B'] == product_id)][['product_name_A','item_A','product_name_B','item_B','confAtoB','lift']].sort_values('lift', ascending = False)
    df = df[df['lift'] > item_lift]
    df = df.sort_values('lift', ascending = False)
    df = df.head(n = num_products)
    product_to_product_associations = df['product_name_A'].values.tolist()
    for x in df['product_name_B'].values.tolist():
        product_to_product_associations.append(x)
    product_to_product_associations = [x for x in product_to_product_associations if x != product_name]
    return product_to_product_associations

In [44]:
def product_recommender(user_id, product_id, item_lift, num_products):
    product_name = products.at[product_id -1,'product_name']
    cluster = clustered_users.at[user_id, 'cluster']
    return product_to_product(cluster = cluster, product_id = product_id, item_lift = item_lift,
                              product_name = product_name, num_products=num_products)

In [45]:
for i in range(0,20):
    user = clustered_users[clustered_users['cluster']==i].sample().index[0]
    print('\ncluster', i, ': user ', user)
    print('\n'.join([x for x in (product_recommender(user, 39055, 1, 5))]))


cluster 0 : user  41229
Uncured Ham & Egg Roasted Potato & Breakfast Buritto
Nacho Sliced Jalapenos
Kale Krunch, Organic, Southwest Ranch
Very Berry Flavor Sparking Mineral Water
Unbleached Recycled Paper Towel Rolls

cluster 1 : user  71672
Roasted Red Pepper Hummus
Mild Guacamole

cluster 2 : user  27046


cluster 3 : user  37788
Indian Paneer Tikka
Corn Tortilla Chips
Organic Raisin Bran Cereal
Organic Mountain Forest Honey Light Amber
Organic Gluten Free Fusilli

cluster 4 : user  68556


cluster 5 : user  30311
Carrot Juice
Simply Organic Garlic Salt
Organic Root Vegetables & Apples with Beef Stage 3 Baby Food
Bourbon Vanilla Bean
Sweet Peas Creamed Spinach Medley

cluster 6 : user  42059
Refried Beans, Black, Fat Free, Organic, Vegetarian
Calcium Gummies
Peanut Butter Chocolate Chip Fruit & Nut Bars
Clover Org Greek Plain
Chocolate Coconut Meal Bar

cluster 7 : user  37131
Green Tea With Ginseng and Honey
Organic Short Grain Brown Rice
Organic Milk Reduced Fat, 2% Milkfat
Organi

In [46]:
product_recommender(22107, 13176, 1, 5)

['Gluten Free Plain Bread',
 'Broccoli Florets, Organic',
 'Shampoo, Body Wash, Bubble Bath Coconut+Lemon',
 'Yellow Peach',
 'Organic Red Anjou Pear']